In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import math

In [4]:
df = pd.read_csv('/Users/joaquingomez/Downloads/dataset/full_dataset.csv')

In [5]:
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [6]:
df.shape

(2231142, 7)

In [8]:
df.isnull().sum()

Unnamed: 0     0
title          0
ingredients    0
directions     0
link           0
source         0
NER            0
dtype: int64

In [9]:
df['title'].unique()

array(['No-Bake Nut Cookies', "Jewell Ball'S Chicken", 'Creamy Corn', ...,
       'Extremely Easy and Quick - Namul Daikon Salad',
       'Pan-Roasted Pork Chops With Apple Fritters',
       'Polpette in Spicy Tomato Sauce'], dtype=object)

In [10]:
df['directions'].unique()

array(['["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]',
       '["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]',
       '["In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings."]',
       ...,
       '["Julienne the daikon and squeeze out the excess moisture.", "Combine all of the ingredients.", "Serve on a dish."]',
       '["In a large bowl, mix the apple cider with 4 cups of water, the sugar and the salt.", "Toast the bay leaves, chilies

In [11]:
# Sample the data for a quick demonstration. 
# I'd suggest using Google Colab, Databricks, Sagemaker or some other cloud sources (with free tier),
# and adjust the size of the data accordingly
n_sample = 10000
df_sample = df.sample(n=n_sample,ignore_index=True)
df_sample.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,756104,Chicken Casserole,"[""1 boned chicken, 6 skinless chicken breasts ...","[""Mix the first 4 ingredients in large cassero...",www.cookbooks.com/Recipe-Details.aspx?id=706889,Gathered,"[""chicken"", ""chicken broth"", ""cream of chicken..."
1,1144182,Root Mousse,"[""2 small rutabagas (young and tender, not hug...","[""Peel and rinse all the root veggies."", ""They...",www.food.com/recipe/root-mousse-167603,Gathered,"[""rutabagas"", ""carrot"", ""potatoes"", ""vegetaria..."
2,967562,Thai Vegetable Noodle Soup My Way,"[""2 tablespoons oil, peanut"", ""1 onion, sweet,...","[""Heat the oil in a work and stir fry the onio...",www.food.com/recipe/thai-vegetable-noodle-soup...,Gathered,"[""oil"", ""onion"", ""garlic"", ""carrot"", ""zucchini..."
3,675217,Uncle Henry'S Cooked Custard Ice Cream,"[""6 eggs"", ""2 c sugar"", ""2 qt. milk"", ""4 t van...","[""Beat eggs and sugar in mixing bowl."", ""Add a...",www.cookbooks.com/Recipe-Details.aspx?id=17354,Gathered,"[""eggs"", ""sugar"", ""milk"", ""vanilla""]"
4,925859,"Field Roast®, Onions, And Kale","[""4 Italian-flavored vegetarian sausage links ...","[""Slice sausage into thick chunks."", ""Coat the...",www.allrecipes.com/recipe/261322/field-roast-o...,Gathered,"[""Italian-flavored vegetarian sausage links"", ..."


In [12]:
df_sample

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,756104,Chicken Casserole,"[""1 boned chicken, 6 skinless chicken breasts ...","[""Mix the first 4 ingredients in large cassero...",www.cookbooks.com/Recipe-Details.aspx?id=706889,Gathered,"[""chicken"", ""chicken broth"", ""cream of chicken..."
1,1144182,Root Mousse,"[""2 small rutabagas (young and tender, not hug...","[""Peel and rinse all the root veggies."", ""They...",www.food.com/recipe/root-mousse-167603,Gathered,"[""rutabagas"", ""carrot"", ""potatoes"", ""vegetaria..."
2,967562,Thai Vegetable Noodle Soup My Way,"[""2 tablespoons oil, peanut"", ""1 onion, sweet,...","[""Heat the oil in a work and stir fry the onio...",www.food.com/recipe/thai-vegetable-noodle-soup...,Gathered,"[""oil"", ""onion"", ""garlic"", ""carrot"", ""zucchini..."
3,675217,Uncle Henry'S Cooked Custard Ice Cream,"[""6 eggs"", ""2 c sugar"", ""2 qt. milk"", ""4 t van...","[""Beat eggs and sugar in mixing bowl."", ""Add a...",www.cookbooks.com/Recipe-Details.aspx?id=17354,Gathered,"[""eggs"", ""sugar"", ""milk"", ""vanilla""]"
4,925859,"Field Roast®, Onions, And Kale","[""4 Italian-flavored vegetarian sausage links ...","[""Slice sausage into thick chunks."", ""Coat the...",www.allrecipes.com/recipe/261322/field-roast-o...,Gathered,"[""Italian-flavored vegetarian sausage links"", ..."
...,...,...,...,...,...,...,...
9995,2027001,STOVE TOP Broccoli Bake,"[""2 eggs"", ""1 can (10 oz./284 mL) condensed cr...","[""Preheat oven to 350F."", ""Beat eggs, soup and...",www.kraftrecipes.com/recipes/stove-top-broccol...,Recipes1M,"[""eggs"", ""condensed cream"", ""Mayonnaise"", ""bro..."
9996,197032,Marinated Asparagus,"[""2 lb. asparagus (can substitute broccoli, st...","[""Pour mixture over asparagus and marinate in ...",www.cookbooks.com/Recipe-Details.aspx?id=620383,Gathered,"[""substitute broccoli"", ""vinegar"", ""sugar"", ""d..."
9997,1586082,Cherimoya Pie,"[""2 cups Cherimoya, Seeded And Pureed*"", ""14-1...","[""Pre-bake pie shell: Poke holes in the crust,...",tastykitchen.com/recipes/desserts/cherimoya-pie/,Gathered,"[""Milk"", ""Sugar"", ""\u00bc"", ""Vanilla"", ""Salt"",..."
9998,510625,Lemon Meringue Pie,"[""1 c. sugar"", ""1/4 tsp. salt"", ""2 eggs, separ...","[""Combine sugar, cornstarch and salt in a doub...",www.cookbooks.com/Recipe-Details.aspx?id=28147,Gathered,"[""sugar"", ""salt"", ""eggs"", ""lemon rind"", ""confe..."


In [14]:
df['directions'][0]

'["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]'

In [13]:
df['directions'][0].replace('"','').replace('[','').replace(']','').replace(',','')

'In a heavy 2-quart saucepan mix brown sugar nuts evaporated milk and butter or margarine. Stir over medium heat until mixture bubbles all over top. Boil and stir 5 minutes more. Take off heat. Stir in vanilla and cereal; mix well. Using 2 teaspoons drop and shape into 30 clusters on wax paper. Let stand until firm about 30 minutes.'

In [16]:
# Apply the cleaning 
df['clean_text'] = [i.replace('"','').replace('[','').replace(']','').replace(',','') for i in df['directions']]
df_sample['clean_text'] = [i.replace('"','').replace('[','').replace(']','').replace(',','') for i in df_sample['directions']]
df_sample = df_sample.drop(columns='Unnamed: 0')
df_sample.head()

,title,ingredients,directions,link,source,NER,clean_text
0,Chicken Casserole,"[""1 boned chicken, 6 skinless chicken breasts ...","[""Mix the first 4 ingredients in large cassero...",www.cookbooks.com/Recipe-Details.aspx?id=706889,Gathered,"[""chicken"", ""chicken broth"", ""cream of chicken...",Mix the first 4 ingredients in large casserole...
1,Root Mousse,"[""2 small rutabagas (young and tender, not hug...","[""Peel and rinse all the root veggies."", ""They...",www.food.com/recipe/root-mousse-167603,Gathered,"[""rutabagas"", ""carrot"", ""potatoes"", ""vegetaria...",Peel and rinse all the root veggies. They need...
2,Thai Vegetable Noodle Soup My Way,"[""2 tablespoons oil, peanut"", ""1 onion, sweet,...","[""Heat the oil in a work and stir fry the onio...",www.food.com/recipe/thai-vegetable-noodle-soup...,Gathered,"[""oil"", ""onion"", ""garlic"", ""carrot"", ""zucchini...",Heat the oil in a work and stir fry the onion ...
3,Uncle Henry'S Cooked Custard Ice Cream,"[""6 eggs"", ""2 c sugar"", ""2 qt. milk"", ""4 t van...","[""Beat eggs and sugar in mixing bowl."", ""Add a...",www.cookbooks.com/Recipe-Details.aspx?id=17354,Gathered,"[""eggs"", ""sugar"", ""milk"", ""vanilla""]",Beat eggs and sugar in mixing bowl. Add a litt...
4,"Field Roast®, Onions, And Kale","[""4 Italian-flavored vegetarian sausage links ...","[""Slice sausage into thick chunks."", ""Coat the...",www.allrecipes.com/recipe/261322/field-roast-o...,Gathered,"[""Italian-flavored vegetarian sausage links"", ...",Slice sausage into thick chunks. Coat the bott...


In [17]:
df_sample.shape

(10000, 7)

In [18]:
from bertopic import BERTopic
# from flair.embeddings import TransformerDocumentEmbeddings

ModuleNotFoundError: No module named 'numba.experimental.structref'